In [ ]:
import pandas as pd
import requests
from collections import defaultdict
from tqdm import tqdm, tqdm_pandas
tqdm.pandas()

In [ ]:
df_officials = pd.DataFrame.from_csv('../RIPE/RIPE_officials_total_actions.csv', index_col=0)

In [ ]:
df_articles = pd.DataFrame(df_officials['article_id'].unique(), columns=['article_id'])

In [ ]:
dict_articles = defaultdict(list)

In [ ]:
%%time
articles = df_articles['article_id'].unique()
for i in range(0,len(articles),20):
    ids = "|".join(map(str,articles[i:i+20]))
    r = requests.get('https://de.wikipedia.org/w/api.php?action=query&pageids={}&prop=categories&format=json&cllimit=max'.format(ids))  
    resp = r.json()
    if "batchcomplete" not in resp:
        print("NOT COMPLETE!")
    pages = resp['query']['pages']
    for page_id,data in pages.items():
        categories = []
        if 'categories' in data:
            for category in data['categories']:
                categories.append(category['title'])
            dict_articles[page_id].extend(categories)

In [ ]:
def politik_in_categories(article_id):
    for cat in dict_articles[str(article_id)]:
        if 'politik' in cat.lower():
            return True
    return False

In [ ]:
df_articles['politik'] = df_articles['article_id'].apply(politik_in_categories)

In [ ]:
df_articles['politik'].value_counts()

In [ ]:
df_articles.to_csv('article_id, category(politik).csv')